## Q1

In [22]:
!du dino_dragon.tflite -h | awk '{ print $1 }'

43M


In [12]:
import tflite_runtime.interpreter as tflite 
import numpy as np 

## Q2

In [7]:
interpreter = tflite.Interpreter(model_path='dino_dragon.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print(f"output index : {output_index}")

output index : 13


## Prepare Image

In [5]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [10]:
interpreter.get_input_details()[0]['shape']

array([  1, 150, 150,   3], dtype=int32)

In [9]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg"
img = download_image(url)
img = prepare_image(img,target_size=(150,150))

/tmp/ipykernel_46054/2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


In [13]:
def prepare_input(x):
    return x / 255.0

In [14]:
x = np.array(img,dtype='float32')
X = np.array([x])
X = prepare_input(X)

In [15]:
X.shape

(1, 150, 150, 3)

## Q3

In [17]:
print(f"First pixel in R : {X[0,0,0,0]}")

First pixel in R : 0.5529412031173706


## Q4 

In [18]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index) 

preds

array([[0.82448506]], dtype=float32)

## Q5

In [21]:
!docker images | grep "svizor42/zoomcamp-dino-dragon-lambda" | awk '{ print $7 }'

639MB
